# Classification Notebook `Duke (Image Recognition)`

## Add JupyterHelper and Download Dependencies

In [6]:
%jars ../libs/JupyterHelper-1.0-SNAPSHOT-jar-with-dependencies.jar

In [7]:
%%loadFromPOM

<properties>
    <deepnetts.version>1.12</deepnetts.version>
    <visrec.version>1.0.1</visrec.version>
</properties>

<dependency>
  <groupId>javax.visrec</groupId>
  <artifactId>visrec-api</artifactId>
  <version>1.0.1</version>
</dependency>
<dependency>
  <groupId>javax.visrec</groupId>
  <artifactId>visrec-ri</artifactId>
  <version>1.0.1</version>
  <type>jar</type>
  <exclusions>
      <exclusion>
          <groupId>com.deepnetts</groupId>
          <artifactId>deepnetts-core</artifactId>
      </exclusion>
  </exclusions>
</dependency>
<dependency>
    <groupId>com.deepnetts</groupId>
    <artifactId>deepnetts-core</artifactId>
    <version>${deepnetts.version}</version>
</dependency>

## Imports

In [8]:
import java.nio.file.*;

import java.io.ObjectInputStream;

import deepnetts.core.DeepNetts;
import deepnetts.data.ImageSet;
import deepnetts.eval.ClassifierEvaluator;
import deepnetts.eval.ConfusionMatrix;
import deepnetts.net.ConvolutionalNetwork;
import deepnetts.net.NeuralNetwork;
import deepnetts.net.layers.activation.ActivationType;
import deepnetts.net.loss.LossType;
import deepnetts.net.train.BackpropagationTrainer;
import deepnetts.util.FileIO;
import javax.visrec.ml.eval.EvaluationMetrics;

import ch.bytecrowd.JupyterHelper

## Download Dataset

In [9]:
var IMAGES_BASEURL   = "https://raw.githubusercontent.com/deepnetts/deepnetts-communityedition/community-visrec/deepnetts-examples/datasets/DukeSet";
var IMAGE_NAMES_URL  = IMAGES_BASEURL + "/train.txt";
var LABELS_NAMES_URL = IMAGES_BASEURL + "/labels.txt";

var IMAGES_NAME_FILE = "/tmp/train.txt";
var LABELS_FILE = "/tmp/labels.txt";

var DUKE_IMAGES_DIR = "/tmp/duke";
var NEGATIVE_IMAGES_DIR = "/tmp/negative";

In [10]:
JupyterHelper.downloadFile(IMAGE_NAMES_URL, IMAGES_NAME_FILE);
JupyterHelper.downloadFile(LABELS_NAMES_URL, LABELS_FILE);

In [11]:
Files.createDirectories(Paths.get(DUKE_IMAGES_DIR));
Files.readAllLines(Paths.get(IMAGES_NAME_FILE)).stream()
    .filter(line -> line.contains("duke"))
    .map(line -> line.replaceAll("duke/| duke", ""))
    .forEach(fileName -> {
        JupyterHelper.downloadFile(IMAGES_BASEURL + "/duke/" + fileName, DUKE_IMAGES_DIR + "/" + fileName);
    });

System.out.println("[+] Files saved to: " + DUKE_IMAGES_DIR);

[+] Files saved to: /tmp/duke


In [13]:
Files.createDirectories(Paths.get(NEGATIVE_IMAGES_DIR));
Files.readAllLines(Paths.get(IMAGES_NAME_FILE)).stream()
    .filter(line -> line.contains("negative"))
    .map(line -> line.replaceAll("negative/| negative", ""))
    .forEach(fileName -> {
        JupyterHelper.downloadFile(IMAGES_BASEURL + "/negative/" + fileName, NEGATIVE_IMAGES_DIR + "/" + fileName);
    });
    
System.out.println("[+] Files saved to: " + NEGATIVE_IMAGES_DIR);

[+] Files saved to: /tmp/negative


## Prepare DataSet

In [8]:
int imageWidth = 64;
int imageHeight = 64;

String trainingFile = IMAGES_NAME_FILE;
String labelsFile = LABELS_FILE;

ImageSet imageSet = new ImageSet(imageWidth, imageHeight);
imageSet.loadLabels(new File(labelsFile));
imageSet.loadImages(new File(trainingFile));
imageSet.zeroMean();
imageSet.setInvertImages(true);
imageSet.shuffle();

Loaded 1 labels
Loaded 73 images


## Splitting Dataset (Train/Test)

In [9]:
System.out.println("[+] Splitting datasets");
var test_train_ratio = 0.7;
var trainTestSet = imageSet.split(test_train_ratio, 1 - test_train_ratio);
System.out.println(String.format("[+] Training data size = %d rows", trainTestSet[0].size()));
System.out.println(String.format("[+] Testing data size  = %d rows", trainTestSet[1].size()));

[+] Splitting datasets
Splitting data set: [0.7, 0.30000000000000004]
[+] Training data size = 51 rows
[+] Testing data size  = 21 rows


## Create a Neuronal Network

In [10]:
System.out.println("[+] Creating the Neural Network");
ConvolutionalNetwork convNet = ConvolutionalNetwork.builder()
    .addInputLayer(imageWidth, imageHeight, 3)
    .addConvolutionalLayer(3, 3, 3, ActivationType.TANH)
    .addMaxPoolingLayer(2, 2, 2)
    .addFullyConnectedLayer(32, ActivationType.TANH)
    .addOutputLayer(1, ActivationType.SIGMOID)
    .lossFunction(LossType.CROSS_ENTROPY)
    .build();

convNet.setOutputLabels(imageSet.getTargetNames());

System.out.println(convNet.toString());

[+] Creating the Neural Network
Input Layer { width:64, height:64, depth:3 }
Convolutional Layer { filter width:3, filter height: 3, channels: 3, stride: 1, activation: TANH}
Max Pooling Layer { filter width:2, filter height: 2, stride:2}
Fully Connected Layer { width:32 activation:TANH}
Output Layer { width:1, activation:SIGMOID}



## Train the Neuronal Network

In [11]:
 BackpropagationTrainer trainer = convNet.getTrainer();
trainer.setMaxError(0.05f)
    .setLearningRate(0.01f);

trainer.train(trainTestSet[0]);

------------------------------------------------------------------------------------------------------------------------------------------------
TRAINING NEURAL NETWORK
------------------------------------------------------------------------------------------------------------------------------------------------
Epoch:1, Time:496ms, TrainError:0.49401402, TrainErrorChange:0.49401402, TrainAccuracy: 0.9019608
Epoch:2, Time:352ms, TrainError:0.29076308, TrainErrorChange:-0.20325094, TrainAccuracy: 0.9411765
Epoch:3, Time:355ms, TrainError:0.23669338, TrainErrorChange:-0.054069698, TrainAccuracy: 0.9411765
Epoch:4, Time:352ms, TrainError:0.20153663, TrainErrorChange:-0.035156757, TrainAccuracy: 0.9411765
Epoch:5, Time:339ms, TrainError:0.17167309, TrainErrorChange:-0.029863536, TrainAccuracy: 0.9607843
Epoch:6, Time:350ms, TrainError:0.14497846, TrainErrorChange:-0.026694626, TrainAccuracy: 0.9607843
Epoch:7, Time:348ms, TrainError:0.121782, TrainErrorChange:-0.023196466, TrainAccuracy: 0

## Evaluating the model

In [12]:
System.out.println("[+] Evaluating the model");
ClassifierEvaluator evaluator = new ClassifierEvaluator();
EvaluationMetrics evalResults = evaluator.evaluate(convNet, trainTestSet[1]);
System.out.println(evalResults);

ConfusionMatrix cm = evaluator.getConfusionMatrix();
System.out.println(cm);

[+] Evaluating the model
TrueNegative: 11.0
Accuracy: 1.0 (How often is classifier correct in total)
TotalClasses: 2.0
FalsePositive: 0.0
FalseNegative: 0.0
Precision: 1.0 (How often is classifier correct when it gives positive prediction)
F1Score: 1.0 (Average of precision and recall)
TruePositive: 10.0
TotalCorrect: 21.0
TotalIncorrect: 0.0
Recall: 1.0 (When it is actually positive class, how often does it give positive prediction)
TotalItems: 21.0

        negativepositive
negative      11       0
positive       0      10



## Save the model

In [13]:
var EXPORT_PATH = "/tmp/DukeClassifier.dnet";
System.out.println("[+] Saving the model to: " + EXPORT_PATH);
FileIO.writeToFile(convNet, EXPORT_PATH);

[+] Saving the model to: /tmp/DukeClassifier.dnet


## Import the model

In [14]:
System.out.println("[+] Importing the model from: " + EXPORT_PATH);
ConvolutionalNetwork neuralNet;
try (ObjectInputStream ois = new ObjectInputStream(new FileInputStream(EXPORT_PATH))) {
    neuralNet = ConvolutionalNetwork.class.cast(ois.readObject()) ;
}

[+] Importing the model from: /tmp/DukeClassifier.dnet
